### This notebook shows a simple analysis of the dutch gas/electricity/heat usage. We will try to get some insights and apply machine learning technique to learn some underlying characteristics of the dataset

In [41]:
import pandas as pd
import numpy as np

import time

import plotly.offline as py
import plotly.graph_objs as go
py.init_notebook_mode(connected=True)

import warnings
warnings.filterwarnings('ignore')

### Load and convert the data into numbers

In [2]:
tabel = pd.read_csv("81528NED_UntypedDataSet_30052019_125411.csv", sep=";")

tabel = tabel.set_index('ID')
for col in ['Woningkenmerken', 'RegioS', 'Perioden']:
    tabel[col] = tabel[col].astype('str')
    
for col in ['GemiddeldAardgasverbruik_1', 'GemiddeldElektriciteitsverbruik_2', 'Stadsverwarming_3']:
    tabel[col] = pd.to_numeric(tabel[col], errors='coerce')

tabel.rename(columns={'Woningkenmerken': 'housing_type', 'RegioS': 'region', 'Perioden': 'period',
                      'GemiddeldAardgasverbruik_1': 'average_gas', 'GemiddeldElektriciteitsverbruik_2': 'average_power',
                      'Stadsverwarming_3': 'city_heat'}, inplace=True)
tabel.head()

,housing_type,region,period,average_gas,average_power,city_heat
ID,,,,,,
0,T001100,NL01,2010JJ00,1850.0,3300.0,4.6
1,T001100,NL01,2011JJ00,1450.0,3250.0,4.5
2,T001100,NL01,2012JJ00,1500.0,3200.0,5.1
3,T001100,NL01,2013JJ00,1600.0,3150.0,4.9
4,T001100,NL01,2014JJ00,1200.0,3050.0,4.8


### Some data visualization

From the graph below, we can clearly see the differences in gas usage among different housing types

In [3]:
chart = go.Scatter(x=tabel['housing_type'], y=tabel['average_gas'], mode='markers', name='gas usage per housing type')
py.iplot([chart])

### Descriptive statistics

In [56]:
tabel.describe()

,average_gas,average_power,city_heat
count,25279.000000,25282.000000,401.000000
mean,1639.839392,3305.614271,14.229676
std,558.376288,781.835511,16.229929
min,150.000000,1450.000000,0.100000
25%,1250.000000,2730.000000,5.000000
50%,1570.000000,3335.000000,7.400000
75%,1950.000000,3800.000000,16.000000
max,5200.000000,6050.000000,71.400000


### Top 5 regions by average gas usage

In [57]:
tabel.groupby('region').mean().dropna().sort_values(by=['average_gas'], ascending=False).head(5)

,average_gas,average_power,city_heat
region,,,
GM0376,2241.000000,3748.166667,7.966667
GM0473,1856.333333,3436.666667,6.525000
GM0362,1808.166667,3600.000000,6.866667
GM0579,1724.666667,3548.500000,9.050000
PV22,1723.000000,3108.333333,0.166667


### Average usage throughout the years

Shows a nice decreasing trend

In [58]:
tabel.groupby('period').mean()
# A decreasing trend in all three types of consumption

,average_gas,average_power,city_heat
period,,,
2010JJ00,2117.926284,3544.843634,18.217143
2011JJ00,1654.754601,3523.466258,18.338235
2012JJ00,1739.229878,3436.855819,15.339130
2013JJ00,1855.104442,3385.481024,15.051111
2014JJ00,1376.615064,3262.578148,14.977273
2015JJ00,1472.644099,3197.591463,12.116667
2016JJ00,1523.023041,3116.428133,12.312308
2017JJ00,1459.110837,3052.330247,12.168182


#### Here, we will use 'housing_type' and 'region' to predict the individual average gas usage. Note that period is excluded because it is not a 'returning feature'. Time series analysis can be done in that regard, but that is for another session.

In [4]:
# One hot encode the categorical variables
dummy_tabel = pd.get_dummies(tabel.drop(['period'], axis=1))
dummy_tabel.head()


,average_gas,average_power,city_heat,housing_type_1014800,housing_type_1014850,housing_type_T001100,housing_type_ZW10300,housing_type_ZW10320,housing_type_ZW25805,housing_type_ZW25806,...,region_PV25,region_PV26,region_PV27,region_PV28,region_PV29,region_PV30,region_PV31,region_PV98,region_PV99,region_PVZZ
ID,,,,,,,,,,,,,,,,,,,,,
0,1850.0,3300.0,4.6,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1450.0,3250.0,4.5,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1500.0,3200.0,5.1,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,1600.0,3150.0,4.9,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,1200.0,3050.0,4.8,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


### Random Forest, Ada Boost and Gradient Boosting

Those are three popular ensemble machine learning techniques to do classification as well as regression. Ensemble learning combines learning algorithms together, in order to improve predictive performance. Random forest uses bagging while the other two use boosting. https://en.wikipedia.org/wiki/Ensemble_learning#Boosting

Those methods will be compared based on their score(which is similar to r-squared in linear regression) and time to train.

In [18]:
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import RandomForestRegressor, AdaBoostRegressor, GradientBoostingRegressor
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split

In [6]:
train_data = dummy_tabel.drop(['average_power', 'city_heat'], axis=1)
train_data = train_data.dropna()
train_x, test_x, train_y, test_y = train_test_split(train_data[train_data.columns[1:]], train_data['average_gas'], test_size=0.1,
                                                    random_state=28)

In [45]:
meths = ['GradientBoostingRegressor', 'AdaBoostRegressor', 'RandomForestRegressor']
compare_table = pd.DataFrame()
n = 200

for meth in meths:
    start = time.time()
    if meth=='RandomForestRegressor':
        rf = RandomForestRegressor(n_estimators=n, verbose=1)
        rf.fit(train_x, train_y)
        compare_table = compare_table.append({"method": meth, "time_taken": time.time()-start, "r-squared": rf.score(test_x, test_y),
                                              "n_estimators": n}, ignore_index=True)
        print("Score for Random forest: %s"%rf.score(test_x, test_y))
    elif meth=='AdaBoostRegressor':
        ada = AdaBoostRegressor(n_estimators=n, learning_rate=1)
        ada.fit(train_x, train_y)
        compare_table = compare_table.append({"method": meth, "time_taken": time.time()-start, "r-squared": ada.score(test_x, test_y),
                                              "n_estimators": n}, ignore_index=True)
        print("Score for Adaptive boosting: %s"%ada.score(test_x, test_y))
    elif meth=='GradientBoostingRegressor':
        gbt = GradientBoostingRegressor(n_estimators=n, learning_rate=1, random_state=0, loss='ls')
        gbt.fit(train_x, train_y)
        compare_table = compare_table.append({"method": meth, "time_taken": time.time()-start, 
                                              "r-squared": gbt.score(test_x, test_y),
                                              "n_estimators": n}, ignore_index=True)
        print("Score for Gradient boosting: %s"%gbt.score(test_x, test_y))
    else:
        print('Please provide a valid method. Choose between RandomForestRegressor, AdaBoostRegressor or GradientBoostingRegressor.')

Score for Gradient boosting: 0.7713715201827456
Score for Adaptive boosting: 0.553776225039065


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 200 out of 200 | elapsed: 16.2min finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 200 out of 200 | elapsed:    0.6s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


Score for Random forest: 0.7716372362338502


[Parallel(n_jobs=1)]: Done 200 out of 200 | elapsed:    0.5s finished


#### While the results of both Gradient Boosting and Random Forest stood out compared to Adaptive boosting, GBT clearly has the advantage in terms of training time. 

In [46]:
compare_table

,method,n_estimators,r-squared,time_taken
0,GradientBoostingRegressor,200.0,0.771372,19.495343
1,AdaBoostRegressor,200.0,0.553776,15.314731
2,RandomForestRegressor,200.0,0.771637,971.727246


#### Here below, we can visualize some relative feature importance. Notice that one housing type contributes more than all other factors combined.

In [40]:
feature_imp = go.Bar(x=test_x.columns, y=gbt.feature_importances_, name='Feature importance')
py.iplot([feature_imp])
                     

### Final thoughts & future work

In this short demo, we showed the application of three machine learning techniques, Random Forest, Adaptive Boosting and Gradient Boosting. The models were able to predict gas usage using predictors like housing type/region, and the scores were fairly good. The following could be done for future improvement:

1. Parameter tuning
2. Explore XGBoost/Catboost, enhanced versions based on Gradient boosting
3. adding more/exploring with other predictors